1. Generate n samples from prompt using AudioLDM and AudioGEN
2. Compute similarity between prompt and MSCLAP / LAION CLAP (perhaps an independent implementation is best)
3. Apply softmax
4. Save torch.topk samples
5. Try k = [1, 8, 64, 512]

Do models based on CLAPs benefit more from this method?

In [9]:
from diffusers import AudioLDMPipeline
from audiocraft.models import AudioGen
import torch
import torchaudio
import laion_clap
import os
import IPython.display as ipd
import json

In [3]:
def set_cuda_device():
    device_name = "CPU"
    if os.name == "posix":
        device = torch.device('mps') if torch.cuda.is_available() else torch.device('cpu')
    if os.name == "nt":
        if (torch.cuda.is_available() != True):
            device = torch.device('cpu')
        else:
            id = torch.cuda.current_device()
            device = torch.device('cuda')
            device_name = torch.cuda.get_device_name(id)

    return device, device_name

def contrastive_reranking(clap_model, audio_model, top_k = 5):
    pass
    # Top k as list

In [16]:
# Parameters
files_to_generate = 1000
seconds_per_file = 5
device = set_cuda_device()
output_dir_audiogen = ".generated_audio/audiogen"
output_dir_audioldm = ".generated_audio/audioldm"

with open('ucs_classes.json', 'r', encoding='utf-8') as f:
    ucs_classes = json.load(f)

In [ ]:
pipe = AudioLDMPipeline.from_pretrained("cvssp/audioldm", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "Techno music with a strong, upbeat tempo and high melodic riffs."
audio = pipe(prompt, num_inference_steps=10, audio_length_in_s=seconds_per_file).audios[0]

In [ ]:

model = AudioGen.get_pretrained('facebook/audiogen-medium')
model.set_generation_params(duration=seconds_per_file)
descriptions = ['dog barking', 'sirene of an emergency vehicle', 'footsteps in a corridor']
wav = model.generate(descriptions)

for idx, one_wav in enumerate(wav):
    # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
    audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
